In [ ]:
from os import path

# Third-party
import astropy.units as u
import h5py
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
from gala.dynamics.mockstream import fardal_stream
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
pot = gp.MilkyWayPotential()
H = gp.Hamiltonian(pot)

In [ ]:
e_factors = np.array([1., 0.84, 0.63, 0.515, 0.09])

# for ef in e_factors:
#     w0 = gd.PhaseSpacePosition(pos=[10, 0, 0]*u.kpc,
#                                vel=[0, 0, 234.2*ef]*u.km/u.s)
#     orbit = pot.integrate_orbit(w0, dt=0.5, t1=0, t2=6*u.Gyr)
    # print(orbit.eccentricity())

In [ ]:
Ms = [1e5] * u.Msun
streams = []
i = 0
for ef in e_factors:
    for m in Ms:
        cache_filename = '../cache/{0}.hdf5'.format(i)
        if path.exists(cache_filename):
            continue
            
        w0 = gd.PhaseSpacePosition(pos=[10, 0, 0]*u.kpc,
                                   vel=[0, 0, 234.2*ef]*u.km/u.s)
        orbit = pot.integrate_orbit(w0, dt=0.5, t1=0, t2=4*u.Gyr)
        
        print('dude', m)
        stream = fardal_stream(H, orbit, m, release_every=8, 
                               snapshot_filename=cache_filename)
        i += 1

In [ ]:
for i in range(3):
    cache_filename = '../cache/{0}.hdf5'.format(i)

    with h5py.File(cache_filename, 'r') as f:
        x, y, z = f['pos'][:]

    fig, ax = plt.subplots(1, 1, figsize=(6, 6))

    j = 0
    l, = ax.plot(x[j, :], z[j, :], 
                 marker='o', ls='none', 
                 ms=3, color='k', alpha=0.2)

    ax.set_xlim(-12, 12)
    ax.set_ylim(-12, 12)
    fig.tight_layout()

    def animate(j):
        l.set_data(x[j, :], z[j, :])
        return l,

    # ani = FuncAnimation(fig, animate, np.arange(0, len(x), 4),
    ani = FuncAnimation(fig, animate, np.arange(4000, 5000, 4),
                        interval=8, blit=True)
    ani.save('/Users/adrian/Downloads/test{}.mp4'.format(i), 
             dpi=150, bitrate=-1)
    del ani